## Description:

Gets and stores data for nodes (representing street segments)

In [2]:
import osmnx as ox
import folium
from geopy.distance import geodesic
from shapely.geometry import Polygon


def filter_edges_by_grid(edges, grid_square):
    print("filtering edges")
    west, south, east, north = grid_square
    grid_polygon = Polygon([(west, south), (west, north), (east, north), (east, south)])
    
    # Use the GeoDataFrame's spatial methods to check if edge geometries intersect with the grid square
    filtered_edges = edges[edges.geometry.apply(lambda geom: geom.intersects(grid_polygon))]
    return filtered_edges


# Function to interpolate points every 10 feet
def interpolate_points(start, end, step_ft):
    points = [start]
    total_distance = geodesic(start, end).feet
    step_ratio = step_ft / total_distance
    while step_ratio < 1.0:
        interp_lat = start[0] + (end[0] - start[0]) * step_ratio
        interp_lon = start[1] + (end[1] - start[1]) * step_ratio
        points.append((interp_lat, interp_lon))
        step_ratio += step_ft / total_distance
    points.append(end)  # Ensure the end point is included
    return points

def create_map_for_edges(edges):
    global grid_counter

    FEET_SPACING
    grid_counter += 1
    print("Creating map for Edges")
    point_counter = 0

    # Initialize a Folium map
    m = folium.Map(location=center_point, zoom_start=16)
    
    # Add edges to the map
    for _, row in edges.iterrows():
        coords = [(y, x) for x, y in row['geometry'].coords]
        
        # Interpolate points between each pair of coordinates
        for i in range(len(coords) - 1):
            start = coords[i]
            end = coords[i + 1]
            interpolated_points = interpolate_points(start, end, step_ft=FEET_SPACING)
            
            # Plot each interpolated point
            for point in interpolated_points:
                point_counter += 1
                # folium.Marker(location=point, popup=f"({location[0]}, {location[1]})").add_to(mymap)
                folium.CircleMarker(
                    location=point,
                    radius=2,
                    color='red',
                    fill=True,
                    fill_opacity=0.8,
                    popup=f"({point[0]}, {point[1]})"
                ).add_to(m)
    
    # Save the map
    m.save(f"data/dc_nodes/dc_{grid_counter}.html")
    print(f"Map created with {point_counter} points.")


def main():

    print("Starting")

    graph = ox.graph_from_place("Washington, DC, USA", network_type='drive')

    print("getting nodes and edges")
    nodes, edges = ox.graph_to_gdfs(graph)

    # prep for batching (since these is way to large of a dataset)
    bbox = edges.total_bounds  # [west, south, east, north]
    print("got bounds")


    # Divide the bounding box into 20 grids (4x5)
    num_rows = 6
    num_cols = 7
    grid_width = (bbox[2] - bbox[0]) / num_cols  # (east - west) / cols
    grid_height = (bbox[3] - bbox[1]) / num_rows  # (north - south) / rows

    grid_squares = []
    print("creating square")
    for row in range(num_rows):
        for col in range(num_cols):
            west = bbox[0] + col * grid_width
            east = west + grid_width
            south = bbox[1] + row * grid_height
            north = south + grid_height
            grid_squares.append((west, south, east, north))

    print(f"grid squares: {len(grid_squares)}")
    for idx, grid_square in enumerate(grid_squares):
        filtered_edges = filter_edges_by_grid(edges, grid_square)
        create_map_for_edges(filtered_edges)

grid_counter = 0
FEET_SPACING = 100
center_point = (38.90251234580012, -77.04350506490759)


main()


Starting
getting nodes and edges
got bounds
creating square
grid squares: 42
filtering edges
Creating map for Edges
Map created with 0 points.
filtering edges
Creating map for Edges
Map created with 0 points.
filtering edges
Creating map for Edges
Map created with 0 points.
filtering edges
Creating map for Edges
Map created with 9400 points.
filtering edges
Creating map for Edges
Map created with 4420 points.
filtering edges
Creating map for Edges
Map created with 0 points.
filtering edges
Creating map for Edges
Map created with 0 points.
filtering edges
Creating map for Edges
Map created with 0 points.
filtering edges
Creating map for Edges
Map created with 0 points.
filtering edges
Creating map for Edges
Map created with 332 points.
filtering edges
Creating map for Edges
Map created with 7409 points.
filtering edges
Creating map for Edges
Map created with 19174 points.
filtering edges
Creating map for Edges
Map created with 11708 points.
filtering edges
Creating map for Edges
Map cre

In [ ]:
pip install folium